In [1]:
# Download the tiny Shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
# Reading the tiny Shakespeare
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
# Number of characters
len(text)

1115394

In [4]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [5]:
# Getting the set of unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


So the model will be character level LLM

In [6]:
for i, ch in enumerate(chars):
    print(i, ' - ', ch)

0  -  

1  -   
2  -  !
3  -  $
4  -  &
5  -  '
6  -  ,
7  -  -
8  -  .
9  -  3
10  -  :
11  -  ;
12  -  ?
13  -  A
14  -  B
15  -  C
16  -  D
17  -  E
18  -  F
19  -  G
20  -  H
21  -  I
22  -  J
23  -  K
24  -  L
25  -  M
26  -  N
27  -  O
28  -  P
29  -  Q
30  -  R
31  -  S
32  -  T
33  -  U
34  -  V
35  -  W
36  -  X
37  -  Y
38  -  Z
39  -  a
40  -  b
41  -  c
42  -  d
43  -  e
44  -  f
45  -  g
46  -  h
47  -  i
48  -  j
49  -  k
50  -  l
51  -  m
52  -  n
53  -  o
54  -  p
55  -  q
56  -  r
57  -  s
58  -  t
59  -  u
60  -  v
61  -  w
62  -  x
63  -  y
64  -  z


In [7]:
# Creating a mapping from charactters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

In [8]:
for c in 'Hello World':
    print(stoi[c])

20
43
50
50
53
1
35
53
56
50
42


In [9]:
# Createing the encoder and decoder
encode = lambda s: [stoi[c] for c in s]
decoder = lambda l: ''.join(([itos[i] for i in l]))

In [10]:
print(encode('Hello World'))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]


So the tokenization code above is our naive way of tokenization for character level

Now we are encoding the entire text dataset and store it into a torch Tensor

In [11]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:300])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

Spliting the dataset

In [13]:
n = int(0.9*len(data))

train_data = data[:n]
val_data =data[n:]

data loader: batches of chunks of data

In [14]:
block_size = 8

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]

    print(f'{context} -> {target}')

tensor([18]) -> 47
tensor([18, 47]) -> 56
tensor([18, 47, 56]) -> 57
tensor([18, 47, 56, 57]) -> 58
tensor([18, 47, 56, 57, 58]) -> 1
tensor([18, 47, 56, 57, 58,  1]) -> 15
tensor([18, 47, 56, 57, 58,  1, 15]) -> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> 58


In [19]:
torch.manual_seed(1337)

batch_size = 4 
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # ix will be a random 4 blocks consist of 8 characters
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('__________')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'{context.tolist()} -> {target}')


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
__________
[24] -> 43
[24, 43] -> 58
[24, 43, 58] -> 5
[24, 43, 58, 5] -> 57
[24, 43, 58, 5, 57] -> 1
[24, 43, 58, 5, 57, 1] -> 46
[24, 43, 58, 5, 57, 1, 46] -> 43
[24, 43, 58, 5, 57, 1, 46, 43] -> 39
[44] -> 53
[44, 53] -> 56
[44, 53, 56] -> 1
[44, 53, 56, 1] -> 58
[44, 53, 56, 1, 58] -> 46
[44, 53, 56, 1, 58, 46] -> 39
[44, 53, 56, 1, 58, 46, 39] -> 58
[44, 53, 56, 1, 58, 46, 39, 58] -> 1
[52] -> 58
[52, 58] -> 1
[52, 58, 1] -> 58
[52, 58, 1, 58] -> 46
[52, 58, 1, 58, 46] -> 39
[52, 58, 1, 58, 46, 39] -> 58
[52, 58, 1, 58, 46, 39, 58] -> 1
[52, 58, 1, 58, 46, 39, 58, 1] -> 46
[25] -> 17
[25, 17] -> 2